In [1]:
import pandas as pd
import numpy as np

from pycta.portfolio import Portfolio
from pycta.beakerx import *

In [2]:
# Load prices
prices = pd.read_csv("data/prices.csv", index_col=0, parse_dates=True).ffill().truncate(before="1970-01-01")

In [3]:
# take two moving averages and apply sign-functiond
def f(price, fast=32, slow=96):
    s = price.ewm(com=slow, min_periods=300).mean()
    f = price.ewm(com=fast, min_periods=300).mean()
    return np.sign(f-s)

In [4]:
form = EasyForm("Configuration")
form.addTextField("fast")
form.addTextField("slow")
form.addButton("Go!", tag="run")
form 

In [5]:
portfolio = Portfolio(prices=prices, position=prices.apply(f, fast=int(form["fast"] or 32), slow=int(form["slow"] or 96)))
analysis(portfolio.nav())

/opt/conda/lib/python3.7/site-packages/beakerx/tabledisplay/tabledisplay.py:109: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.
  value = args[0][columnName].get_values()[index]
/opt/conda/lib/python3.7/site-packages/beakerx/tabledisplay/tabledisplay.py:114: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.
  index_values = args[0].index.get_values()[index]


* This is a **univariate** trading system, we map the (real) price of an asset to its (cash)position.
* Some hedge funds would argue that they are running 49 trading systems.
* Only 2 **free parameters** used here.
* Only 3 lines of code.

Results do not look terrible but...
* No concept of risk integrated. 
* The size of each bet is constant regardless of the underlying asset.
* The system lost its mojo in 2009 and has never really recovered.
* The sign function is very expensive to trade as position changes are too extreme.

Such fundamental flaws are not addressed by **parameter-hacking** or **pimp-my-trading-system** steps (remove the worst performing assets, insane quantity of stop-loss limits, ...)